In [ ]:
!pip install diffusers transformers accelerate flask flask-cors pyngrok torch torchvision

In [ ]:
import os
os.environ['NGROK_AUTH_TOKEN'] = '34mP4pTTHmzflKgiWPHNI3LANG4_2mJdyENQ8JCPaPHpcfwwB'

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from diffusers import (
    StableDiffusionXLPipeline,
    StableDiffusionPipeline,
    DPMSolverMultistepScheduler,
    EulerAncestralDiscreteScheduler
)
import torch
from PIL import Image
import io
import base64
from pyngrok import ngrok
import os


In [ ]:
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})


In [ ]:
# Handle old model names for backward compatibility
def load_model(model_name):
    if model_name == "sdxl-1.5":
        model_name = "stable-diffusion-xl"
    elif model_name == "realistic-vision":
        model_name = "dreamshaper"

    device = "cuda" if torch.cuda.is_available() else "cpu"

    if model_name == "stable-diffusion-v1-5":
        # Stable Diffusion v1.5
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            safety_checker=None
        )
        pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

    elif model_name == "dreamshaper":
        # DreamShaper v8
        pipe = StableDiffusionPipeline.from_pretrained(
            "Lykon/DreamShaper",
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            safety_checker=None
        )
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

    elif model_name == "stable-diffusion-xl":
        # SDXL Base 1.0
        pipe = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            variant="fp16" if device == "cuda" else None
        )

    else:
        raise ValueError(f"Unknown model: {model_name}")

    pipe = pipe.to(device)

    # Enable memory optimizations
    if device == "cuda":
        pipe.enable_attention_slicing()
        pipe.enable_vae_slicing()
        # Use xformers if available
        try:
            pipe.enable_xformers_memory_efficient_attention()
        except:
            pass

    loaded_models[model_name] = pipe
    current_model = pipe

    print(f"Model {model_name} loaded successfully!")
    return pipe


def load_lora(pipe, lora_name):
    """Load LoRA weights"""
    if not lora_name or lora_name == "":
        return pipe

    try:
        print(f"Loading LoRA: {lora_name}")

        if lora_name == "realistic-vision-v6":
            # Realistic Vision V6.0 B1 HyperVAE
            pipe.load_lora_weights(
                "SG161222/Realistic_Vision_V6.0_B1_noVAE",
                weight_name="realisticVisionV60B1_v51HyperVAE.safetensors"
            )

        elif lora_name == "fearii-v1.1":
            # Fearii LoHA v1.1
            pipe.load_lora_weights(
                "Shalie/Fearii",
                weight_name="fearii-loha-v1.1.safetensors"
            )

        print(f"LoRA {lora_name} loaded successfully!")
    except Exception as e:
        print(f"Warning: Could not load LoRA {lora_name}: {e}")
        print("Continuing without LoRA...")

    return pipe


@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'gpu_available': torch.cuda.is_available(),
        'loaded_models': list(loaded_models.keys())
    })


@app.route('/generate', methods=['POST'])
def generate():
    """Generate image based on the request"""
    try:
        data = request.json
        model_name = data.get('model', 'stable-diffusion-v1-5')
        lora_name = data.get('lora', '')
        prompt = data.get('prompt', '')
        seed = data.get('seed', 0)

        if not prompt:
            return jsonify({'error': 'Prompt is required'}), 400

        # Load the model
        pipe = load_model(model_name)

        # Load LoRA if specified
        if lora_name:
            pipe = load_lora(pipe, lora_name)

        # Set the seed for reproducibility
        generator = torch.Generator(device=pipe.device).manual_seed(seed)

        # Generate the image with better settings
        print(f"Generating image with prompt: {prompt[:50]}...")

        if model_name == "stable-diffusion-xl":
            # SDXL settings for higher quality
            image = pipe(
                prompt=prompt,
                generator=generator,
                num_inference_steps=40,
                guidance_scale=7.5,
                width=1024,
                height=1024
            ).images[0]
        else:
            # SD 1.5 and DreamShaper settings
            image = pipe(
                prompt=prompt,
                generator=generator,
                num_inference_steps=30,
                guidance_scale=7.5,
                width=768,
                height=768
            ).images[0]

        # Convert image to base64
        buffered = io.BytesIO()
        image.save(buffered, format="PNG", quality=95)
        img_str = base64.b64encode(buffered.getvalue()).decode()

        print("Image generated successfully!")

        return jsonify({
            'success': True,
            'image': f'data:image/png;base64,{img_str}',
            'seed': seed
        })

    except Exception as e:
        print(f"Error generating image: {str(e)}")
        import traceback
        traceback.print_exc()
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500


def start_server(port=7860):
    """Start the Flask server with ngrok tunnel"""
    # Set your ngrok auth token here
    ngrok_token = os.environ.get('NGROK_AUTH_TOKEN', '')

    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)

    # Start ngrok tunnel
    public_url = ngrok.connect(port)

    print("\n" + "=" * 60)
    print("🚀 Colab Backend Started!")
    print("=" * 60)
    print(f"\n🔗 Public URL: {public_url.public_url}")
    print(f"\n📋 Use this URL: {public_url.public_url}")
    print("\n" + "=" * 60 + "\n")

    # Run Flask app
    app.run(port=port, host='0.0.0.0')


if __name__ == '__main__':
    # Pre-load the default model (DreamShaper for best quality)
    print("Pre-loading default model (DreamShaper)...")
    load_model("dreamshaper")

    # Start the server
    start_server()


Pre-loading default model (DreamShaper)...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--Lykon--DreamShaper/snapshots/228d79cb20811466f5c5710aa91f05dabd0b8a14/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--Lykon--DreamShaper/snapshots/228d79cb20811466f5c5710aa91f05dabd0b8a14/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--Lykon--DreamShaper/snapshots/228d79cb20811466f5c5710aa91f05dabd0b8a14/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--Lykon--DreamShaper/snapshots/228d79cb20811466f5c5710aa91f05dabd0b8a14/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by p

Model dreamshaper loaded successfully!

🚀 Colab Backend Started!

🔗 Public URL: https://b8b87346f199.ngrok-free.app

📋 Use this URL: https://b8b87346f199.ngrok-free.app


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7860
 * Running on http://172.28.0.12:7860
INFO:werkzeug:Press CTRL+C to quit
